In [1]:
import pandas as pd
import numpy as np

In [3]:
DATA_PATH = './data/'

users = pd.read_csv(DATA_PATH + 'users_preprocessed.csv')
books = pd.read_csv(DATA_PATH + 'books_merged.csv')
train = pd.read_csv(DATA_PATH + 'train_ratings.csv')
test = pd.read_csv(DATA_PATH + 'test_ratings.csv')
sub = pd.read_csv(DATA_PATH + 'sample_submission.csv')

In [4]:
features_name = ['isbn']

context_df = train.merge(users, on='user_id', how='left').merge(books[features_name], on='isbn', how='left')
train_df = test.merge(users, on='user_id', how='left').merge(books[features_name], on='isbn', how='left')
test_df =sub.merge(users, on='user_id', how='left').merge(books[features_name], on='isbn', how='left')

In [5]:
context_df

,user_id,isbn,rating,age,location_city,location_state,location_country
0,8,0002005018,4,18.0,ontario_others,ontario,canada
1,67544,0002005018,7,30.0,toronto,ontario,canada
2,123629,0002005018,8,30.2,kingston,ontario,canada
3,200273,0002005018,8,18.0,ontario_others,ontario,canada
4,210926,0002005018,9,26.5,guelph,ontario,canada
...,...,...,...,...,...,...,...
306790,278843,0743525493,7,28.0,california_others,california,usa
306791,278851,067161746X,6,33.0,dallas,texas,usa
306792,278851,0884159221,7,33.0,dallas,texas,usa
306793,278851,0912333022,7,33.0,dallas,texas,usa


In [12]:
train['isbn'].nunique()

129777

In [14]:
# 책 이름을 기준으로 평균 평점을 merge
mean_ratings = train.groupby('isbn')['rating'].mean().to_frame()
count_ratings = train.groupby('isbn')['rating'].count().to_frame()

In [19]:
print(len(mean_ratings))
print(len(count_ratings))

129777
129777


In [17]:
isbn_mean_count = mean_ratings.merge(count_ratings, on='isbn', how='left')

In [21]:
isbn_mean_count.columns = ['mean_rating', 'count_rating']

In [22]:
isbn_mean_count

,mean_rating,count_rating
isbn,,
0000913154,8.0,1
000104687X,6.0,1
0001047213,8.0,1
0001047973,9.0,2
000104799X,6.0,1
...,...,...
B0001FZGBC,7.0,1
B0001FZGPI,6.0,1
B0001FZGRQ,9.0,1


In [31]:
# 평균 평점이 3점 이하이고, 3개 이상의 count인 책들 집계
bad_books = isbn_mean_count[(isbn_mean_count['mean_rating'] <= 3.0) & (isbn_mean_count['count_rating'] >= 3.0)]

In [35]:
bad_books.sample(1)

,mean_rating,count_rating
isbn,,
0373031289,3.0,3


In [32]:
len(bad_books)

142

In [ ]:
# bad books로 집계된 책들이 train에 얼마나 있을까?


In [33]:
# 평균 평점이 3점 이하이고, 3개 이상의 count인 책들 집계
good_books = isbn_mean_count[(isbn_mean_count['mean_rating'] >= 8.0) & (isbn_mean_count['count_rating'] >= 3.0)]

In [34]:
len(good_books)

1381

In [36]:
good_books.sample(1)

,mean_rating,count_rating
isbn,,
0517799693,9.666667,3


In [45]:
# 평균 평점이 n점 이하이고, 3개 이상의 count인 책들 집계
bad_books_dict = dict()
for n in range(1,5):
    bad_books_dict[n] = isbn_mean_count[(isbn_mean_count['mean_rating'] <= n) & (isbn_mean_count['count_rating'] >= 3.0)]
    print(f"평균 평점이 {n}점 이하이고, 3개 이상의 count인 책들 집계")
    print(len(bad_books_dict[n]))
    print((bad_books_dict[n]['count_rating']).sum())
    if n == 1:
        print(f"구간별 count: {(bad_books_dict[n]['count_rating']).sum()}")
        tmp = (bad_books_dict[n]['count_rating']).sum()
    else:
        print(f"구간별 count: {(bad_books_dict[n]['count_rating']).sum() - tmp}")
        tmp = (bad_books_dict[n]['count_rating']).sum()

평균 평점이 1점 이하이고, 3개 이상의 count인 책들 집계
4
12
구간별 count: 12
평균 평점이 2점 이하이고, 3개 이상의 count인 책들 집계
29
98
구간별 count: 86
평균 평점이 3점 이하이고, 3개 이상의 count인 책들 집계
142
464
구간별 count: 366
평균 평점이 4점 이하이고, 3개 이상의 count인 책들 집계
568
2516
구간별 count: 2052


In [49]:
# 평균 평점이 n점 이하이고, 3개 이상의 count인 책들 집계
good_books_dict = dict()
for n in range(10, 5, -1):
    good_books_dict[n] = isbn_mean_count[(isbn_mean_count['mean_rating'] >= n) & (isbn_mean_count['count_rating'] >= 3.0)]
    print(f"평균 평점이 {n}점 이상이고, 3개 이상의 count인 책들 집계")
    print(len(good_books_dict[n]))
    print((good_books_dict[n]['count_rating']).sum())
    if n == 10:
        print(f"구간별 count: {(good_books_dict[n]['count_rating']).sum()}")
        tmp = (good_books_dict[n]['count_rating']).sum()
    else:
        print(f"구간별 count: {(good_books_dict[n]['count_rating']).sum() - tmp}")
        tmp = (good_books_dict[n]['count_rating']).sum()

평균 평점이 10점 이상이고, 3개 이상의 count인 책들 집계
54
169
구간별 count: 169
평균 평점이 9점 이상이고, 3개 이상의 count인 책들 집계
1381
5884
구간별 count: 5715
평균 평점이 8점 이상이고, 3개 이상의 count인 책들 집계
6338
39908
구간별 count: 34024
평균 평점이 7점 이상이고, 3개 이상의 count인 책들 집계
13511
115508
구간별 count: 75600
평균 평점이 6점 이상이고, 3개 이상의 count인 책들 집계
18696
161268
구간별 count: 45760
